In [1]:
!pip install obs-websocket-py
!pip install playwright
from playwright.async_api import async_playwright
from obswebsocket import obsws, requests
import time

In [2]:
OBS_HOST = "192.168.1.6"
OBS_PORT = 4444  # Thay đổi nếu bạn cấu hình cổng khác
OBS_PASSWORD = "Abc@1234"  # Thay đổi bằng mật khẩu của bạn

In [10]:
ws = obsws(OBS_HOST, OBS_PORT, OBS_PASSWORD)
ws.connect()

In [16]:
ws.call(requests.StartRecording()) 
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False, args=["--window-name=PlaywrightWindow"])
    page = await browser.new_page()
    await page.goto("https://example.com")  # Đi đến trang web cần kiểm thử
    await  page.wait_for_timeout(25000)  # Đợi 5 giây để quan sát
    #await browser.close()
    ws.call(requests.StopRecording())  # Dừng ghi OBS


In [6]:
ws.disconnect()  # Ngắt kết nối OBS WebSocket

In [22]:

async def main():
    # Kết nối đến OBS WebSocket
    ws = obsws(OBS_HOST, OBS_PORT, OBS_PASSWORD)
    ws.connect()

    try:
        # Khởi động Playwright và mở trình duyệt
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            await page.goto("https://example.com")  # Đi đến trang web cần kiểm thử


            # Lấy danh sách các cửa sổ hiện có trong OBS
            scenes = ws.call(requests.GetSceneList())
            print(scenes)
            current_scene = scenes.getScenes()[0]["name"]  # Lấy tên scene hiện tại

            # Xóa tất cả các nguồn cũ trong scene
            sources = ws.call(requests.GetSceneItemList(sceneName=current_scene))
            for source in sources.getSceneItems():
                ws.call(requests.RemoveSceneItem(sceneName=current_scene, item=source["sourceName"]))

            # Thêm nguồn "Window Capture" mới
            window_title = "Chrominum"  # Tiêu đề cửa sổ trình duyệt của Playwright
            ws.call(requests.CreateSource(
                sourceName="Playwright Window",
                sourceKind="Window Capture (PipeWire)",
                sceneName=current_scene,
                sceneItemEnabled=True
            ))
            ws.call(requests.SetSceneItemProperties(
                sceneName=current_scene,
                item="Playwright Window",
                visible=True
            ))
            ws.call(requests.SetSourceSettings(
                sourceName="Playwright Window",
                sourceSettings={"window": window_title}
            ))

            print("Bắt đầu ghi OBS...")
            ws.call(requests.StartRecording())  # Bắt đầu ghi OBS

            # Thực hiện các hành động trên trang web
            await page.wait_for_timeout(10000)  # Đợi 10 giây để quan sát

            print("Kết thúc ghi OBS...")
            ws.call(requests.StopRecording())  # Dừng ghi OBS

            # Đóng trình duyệt sau khi ghi xong
            #await browser.close()

    finally:
        ws.disconnect()  # Ngắt kết nối OBS WebSocket

# Chạy hàm chính
await main()

<GetSceneList request ({}) called: success ({'message-id': '3', 'status': 'ok', 'current-scene': 'Scene', 'scenes': [{'name': 'Scene', 'sources': []}, {'name': 'Vertical Scene', 'sources': []}]})>
Bắt đầu ghi OBS...
Kết thúc ghi OBS...


In [53]:
async def main():
    # Kết nối đến OBS WebSocket 
    # <<< LỖI: Cần client = obsws(...) và await client.connect() >>>
    ws = obsws(OBS_HOST, OBS_PORT, OBS_PASSWORD) 
    ws.connect() 

    try:
        # Khởi động Playwright và mở trình duyệt
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=False)
            page = await browser.new_page()
            await page.goto("https://example.com")  
            page_title = await page.title() # Lấy title thực tế

            # <<< LỖI: Tất cả ws.call(...) bên dưới đều thiếu await >>>

            # Lấy danh sách scenes 
            scenes = ws.call(requests.GetSceneList()) 
            print(scenes) 
            # Giả sử lấy scene đầu tiên
            current_scene = scenes.getScenes()[0]["name"] # <<< Sửa key thành sceneName nếu cần

            # Xóa nguồn cũ (Đã cảnh báo là nguy hiểm và sai tham số)
            # sources = ws.call(requests.GetSceneItemList(sceneName=current_scene))
            # for source in sources.getSceneItems():
            #     ws.call(requests.RemoveSceneItem(sceneName=current_scene, item=source["sourceName"])) # <<< LỖI: Tham số item sai

            # --- Tập trung vào các tham số theo yêu cầu ---

            # 1. Tham số cho CreateSource
            #    =========================
            #    <<< LƯU Ý QUAN TRỌNG về sourceKind >>>
            #    Giá trị "Window Capture (PipeWire)" có thể không chính xác.
            #    Bạn cần kiểm tra trong OBS xem tên Kind thực tế là gì khi thêm thủ công.
            #    Thử các giá trị như: "window_capture", "pipewire-window-capture", "browser_source", v.v.
            VERIFIED_SOURCE_KIND = "window_capture" # <<< THAY THẾ BẰNG KIND ĐÚNG TỪ OBS
            
            ws.call(requests.CreateSource(
                sourceName="Playwright Window",      # Tên bạn muốn đặt cho source
                sourceKind=VERIFIED_SOURCE_KIND,     # <<< Dùng Kind đã xác minh ở trên
                sceneName=current_scene,             # Scene đích
                # sceneItemEnabled=True              # <<< Tham số này có thể không cần, nên dùng SetSceneItemEnabled sau
                setVisible=True                      # <<< Hoặc thử setVisible=True ở đây
            ))

            # 2. Tham số cho SetSceneItemProperties (hoặc SetSceneItemEnabled)
            #    =============================================================
            #    Dùng để đảm bảo source hiển thị sau khi tạo/cập nhật
            ws.call(requests.SetSceneItemProperties(
                sceneName=current_scene,
                item="Playwright Window",  # <<< Dùng tên sourceName đã tạo. OBS sẽ tìm theo tên.
                visible=True              # Tham số 'visible' là đúng
            ))
            # Hoặc dùng SetSceneItemEnabled nếu chỉ muốn bật/tắt:
            # ws.call(requests.SetSceneItemEnabled(
            #      sceneName=current_scene,
            #      sceneItemId= ??? , # <<< Cần ID của item, lấy từ GetSceneItemList sau khi tạo
            #      sceneItemEnabled=True
            # ))


            # 3. Tham số cho SetSourceSettings
            #    =============================
            #    <<< LƯU Ý QUAN TRỌNG về sourceSettings["window"] >>>
            #    - Key "window" có đúng cho VERIFIED_SOURCE_KIND bạn dùng không? Cần kiểm tra.
            #    - Giá trị của nó KHÔNG PHẢI là "Chrominum" hay chỉ title trang web.
            #    - Nó phải là chuỗi định danh cửa sổ CHÍNH XÁC mà OBS hiển thị.
            WINDOW_IDENTIFIER_PLACEHOLDER = "[chromium.chrome]: Example Domain - Chromium" # <<< THAY THẾ BẰNG ĐỊNH DANH ĐÚNG TỪ OBS
            
            # Thêm delay nhỏ trước khi cài đặt settings (quan trọng)
            await asyncio.sleep(3) # <<< Cần asyncio và await ở đây!

            ws.call(requests.SetSourceSettings(
                sourceName="Playwright Window",      # Tên source đã tạo
                sourceSettings={
                    "window": WINDOW_IDENTIFIER_PLACEHOLDER # <<< Dùng định danh đúng đã tìm thấy
                    # Có thể có các settings khác tùy thuộc vào sourceKind
                 } 
            ))

            # --- Kết thúc phần tập trung vào tham số ---

            print("Bắt đầu ghi OBS...")
            ws.call(requests.StartRecording()) # <<< LỖI: Thiếu await

            await page.wait_for_timeout(10000) 

            print("Kết thúc ghi OBS...")
            ws.call(requests.StopRecording()) # <<< LỖI: Thiếu await

            # Đóng trình duyệt
            # await browser.close()

    finally:
        # <<< LỖI: Thiếu await >>>
        ws.disconnect() 

# Chạy hàm chính
# <<< Cần try/except và import asyncio nếu dùng await main() >>>
await main()

<GetSceneList request ({}) called: success ({'message-id': '3', 'status': 'ok', 'current-scene': 'Scene', 'scenes': [{'name': 'Scene', 'sources': [{'name': 'Window Capture (PipeWire)', 'id': 14, 'type': 'pipewire-window-capture-source', 'volume': 1.0, 'x': 0.0, 'y': 0.0, 'source_cx': 0, 'source_cy': 0, 'muted': False, 'alignment': 5, 'cx': 0.0, 'cy': 0.0, 'render': True, 'locked': False}]}, {'name': 'Vertical Scene', 'sources': []}]})>
Bắt đầu ghi OBS...
Kết thúc ghi OBS...


In [34]:
ws.disconnect()

In [54]:
import obspython

ModuleNotFoundError: No module named 'obspython'

Connection lost!
Connection lost!
Connection lost!
Connection lost!
Connection lost!
